# 🔹 LLM Train Loop Teorisi

LLM'lerde **train loop**, modelin veriyi öğrenmesini sağlayan çekirdek mekanizmadır. Burada **compile**, **metric** veya **validation** kısmına girmeden sadece **forward, loss ve backward** adımlarını ele alıyoruz.


## 1️⃣ Veri Hazırlığı ve Batch İşleme
- Veriler **mini-batch** hâlinde işlenir:
  - `input_ids` → Encoder/Decoder girişleri
  - `target_ids` → Öğrenilecek hedef token dizileri
- **Amaç:** GPU hafızasını korumak ve öğrenmeyi stabil hâle getirmek.

> Mini-batch, hem hesaplama verimliliği sağlar hem de gradyanların daha stabil olmasına yardımcı olur.


## 2️⃣ Forward Pass
- Model, ağırlıklar üzerinden veri geçirir ve **tahminler (logits)** üretir.
- **Decoder** için genellikle **teacher forcing** uygulanır:
  - Decoder’a bir adım kaydırılmış hedef tokenlar verilir.
  - Hataların birikmesi engellenir, öğrenme hızlanır.
- Çıkış boyutu: `[batch_size, seq_len, vocab_size]`


## 3️⃣ Loss Hesaplama
- Tahminler ile gerçek hedefler arasındaki fark ölçülür.
- Genellikle **CrossEntropyLoss** kullanılır:
  - Padding tokenlar loss hesabına dahil edilmez (`ignore_index=PAD_TOKEN`)
- Tensor boyutları reshape edilir: `[B*T, V]` ve `[B*T]`

> Model her token için doğru kelimeyi tahmin etmeye çalışır. Loss, token’ların ortalaması olarak hesaplanır.


## 4️⃣ Backward Pass ve Optimize
- `loss.backward()` → Gradyanlar hesaplanır (chain rule)
- **Gradient clipping** → Çok derin ağlarda gradyan patlamasını önler
- `optimizer.step()` → Model ağırlıkları güncellenir

> Forward’da hesaplanan hatayı geriye yayarak model daha doğru tahminler yapmayı öğrenir.


## 5️⃣ Epoch Döngüsü
- Her **epoch**, tüm veri üzerinden bir geçiştir.
- Genellikle epoch sonunda:
  - Ortalama loss hesaplanır
  - Modelin öğrenme durumu gözlemlenir

> Validation veya metric hesaplama burada işlenmez; sadece **train loop çekirdeği** odaklanılır.



## 🔑 Özet Mantık
1. **Batch al** → Veri küçük parçalara bölünür  
2. **Forward** → Model tahmin yapar (teacher forcing)  
3. **Loss** → Tahmin ile gerçek hedef karşılaştırılır  
4. **Backward** → Gradyanlar hesaplanır  
5. **Optimize** → Ağırlıklar güncellenir  

> Bu 5 adım, tüm LLM train loop’larının temelini oluşturur.


----

# LLM Train Loop:
```bash

Start Epoch
   │
   ▼
Load Mini-Batch
   │
   ▼
Move input_ids & target_ids to Device
   │
   ▼
Forward Pass (model(input_ids, target_ids))
   │
   ▼
Compute Loss (CrossEntropyLoss)
   │
   ▼
Backward Pass (loss.backward())
   │
   ▼
Gradient Clipping (optional)
   │
   ▼
Optimizer Step (update model weights)
   │
   ▼
Accumulate total_loss
   │
   ▼
More Batches? ── Yes ──▶ Back to Load Mini-Batch
      │
      No
      │
      ▼
Compute Average Loss for Epoch
      │
      ▼
More Epochs? ── Yes ──▶ Back to Load Mini-Batch
      │
      No
      │
      ▼
End Training Loop
